In [1]:
# Instalar e importar bibliotecas
!pip install ipywidgets
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Audio
import scipy.io.wavfile as wavfile
import io

# Função para converter dados em áudio

def data_to_audio_wave(data, duration_per_note=0.4, sample_rate=44100):
    note_freqs = [
        261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88,
        523.25, 587.33, 659.25, 698.46, 783.99, 880.00, 987.77
    ]
    values = pd.Series(data).dropna().astype(float)
    min_val, max_val = values.min(), values.max()
    norm = ((values - min_val) / (max_val - min_val) * (len(note_freqs) - 1)).astype(int)
    freqs = [note_freqs[i] for i in norm]

    audio = np.array([], dtype=np.float32)
    for f in freqs:
        t = np.linspace(0, duration_per_note, int(sample_rate * duration_per_note), endpoint=False)
        wave = 0.5 * np.sin(2 * np.pi * f * t)
        audio = np.concatenate([audio, wave])

    audio_int16 = np.int16(audio / np.max(np.abs(audio)) * 32767)
    return sample_rate, audio_int16

# Widgets e lógica interativa

upload = widgets.FileUpload(accept='.csv', multiple=False)
x_dropdown = widgets.Dropdown(description='Eixo X (tempo):')
y_dropdown = widgets.Dropdown(description='Eixo Y (valor):')
play_button = widgets.Button(description='Tocar Música')
output_area = widgets.Output()


def is_temporal_or_sequential(series: pd.Series) -> bool:
    name = series.name.lower()
    if any(k in name for k in ["year", "date", "month", "day", "time"]):
        return True
    if pd.api.types.is_numeric_dtype(series) and series.nunique() > 5:
        unique_vals = series.dropna().unique()
        unique_vals.sort()
        diffs = np.diff(unique_vals)
        if np.all(diffs > 0):
            return True
    return False


def handle_upload(change):
    output_area.clear_output()
    if upload.value:
        content = list(upload.value.values())[0]['content']
        df = pd.read_csv(io.BytesIO(content))
        numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        temporal_cols = [col for col in df.columns if is_temporal_or_sequential(df[col])]

        x_dropdown.options = temporal_cols
        y_dropdown.options = numeric_cols
        x_dropdown.value = temporal_cols[0] if temporal_cols else None
        y_dropdown.value = numeric_cols[0] if numeric_cols else None

        play_button.df = df

        with output_area:
            display(df.head())


def handle_play(b):
    output_area.clear_output()
    df = b.df
    x_col = x_dropdown.value
    y_col = y_dropdown.value

    if x_col and y_col:
        with output_area:
            plt.figure(figsize=(10, 4))
            plt.plot(df[x_col], df[y_col])
            plt.title(f'{y_col} ao longo de {x_col}')
            plt.xlabel(x_col)
            plt.ylabel(y_col)
            plt.grid(True)
            plt.tight_layout()
            plt.show()

            sample_rate, audio_data = data_to_audio_wave(df[y_col])
            wav_io = io.BytesIO()
            wavfile.write(wav_io, sample_rate, audio_data)
            wav_io.seek(0)
            display(Audio(wav_io.read(), rate=sample_rate))


upload.observe(handle_upload, names='value')
play_button.on_click(handle_play)

# Exibir interface

display(widgets.VBox([upload, x_dropdown, y_dropdown, play_button, output_area]))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00
